## Hybrid Search Qdrant and Memory Integration = LlamaIndex

LLM Usage - Gemini 1.5 Flash

### Installation

In [ ]:
!pip install llama-index llama-index-llms-gemini llama-index-vector-stores-qdrant fastembed
!pip install llama-index-embeddings-fastembed

## Setup LLM and Embedding model

In [ ]:
import os
from getpass import getpass
from llama_index.llms.gemini import Gemini

GOOGLE_API_KEY = getpass("Enter your Gemini API:")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = Gemini() # gemini 1.5 flash

Enter your Gemini API:··········


In [ ]:
llm.complete("when was One Piece started?").text

'One Piece was first serialized in Weekly Shōnen Jump on **July 22, 1997**. \n'

In [ ]:
from llama_index.core import Settings

Settings.llm = llm

In [ ]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding

Settings.embed_model = FastEmbedEmbedding() # thenlper/gte-large

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

## Load your data

In [ ]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/").load_data()

In [ ]:
documents

## Define Qdrant in memory, keep enable_hybrid=True

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

In [ ]:
client = qdrant_client.QdrantClient(
    location=":memory:",
)

vector_store = QdrantVectorStore(
    collection_name = "paper",
    client=client,
    enable_hybrid=True, # Hybrid Search will take place
    batch_size=20,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

## Index your document

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

## Run User Query -> Enable Hybrid mode

In [ ]:
query_engine = index.as_query_engine(
    vector_store_query_mode="hybrid"
)

In [ ]:
response = query_engine.query("what is the meaning of life?")
print(response)

The provided text focuses on the use of Large Language Models (LLMs) for planning in autonomous agents. It does not discuss the meaning of life. 



In [ ]:
llm.complete("what is the meaning of life?").text

"The meaning of life is a question that has been pondered by philosophers, theologians, and individuals for centuries. There is no single, universally accepted answer, as the meaning of life is deeply personal and subjective. \n\nHere are some perspectives on the meaning of life:\n\n**Philosophical Perspectives:**\n\n* **Nihilism:**  Life has no inherent meaning or purpose.\n* **Existentialism:**  Individuals create their own meaning through their choices and actions.\n* **Absurdism:**  Life is inherently meaningless, but we can find meaning in embracing the absurdity.\n* **Hedonism:**  The pursuit of pleasure and happiness is the ultimate goal.\n* **Utilitarianism:**  The greatest good for the greatest number of people is the ultimate goal.\n\n**Religious Perspectives:**\n\n* **Theism:**  Life has meaning and purpose as defined by a higher power or deity.\n* **Buddhism:**  The goal of life is to achieve enlightenment and escape the cycle of suffering.\n* **Christianity:**  The meaning

In [ ]:
response = query_engine.query("give the abstract within 2 sentence")
print(response)

This document explores the use of large language models (LLMs) as agents for solving complex tasks. It focuses on two main approaches: decomposition-first methods, where the task is broken down into sub-tasks before execution, and interleaved decomposition methods, which dynamically adjust the decomposition based on feedback. 



### Using Chat Memory Buffer and Using Context as Chat_mode for Chat Engine

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3000)

In [ ]:
chat_engine = index.as_chat_engine(
    chat_mode="context", # rag based application where we have outside or external data make sure the chat mode is context
    memory=memory,
    system_prompt=(
        "You are an AI assistant who answer the user questions"
    ),
)

In [ ]:
from IPython.display import Markdown, display

In [ ]:
check1 = chat_engine.chat("give the abstract within 2 sentence")

In [ ]:
display(Markdown(f"<b>{check1}</b>"))

<b>This paper explores the use of large language models (LLMs) as agents in various environments, including question-answering and interactive programming. The authors evaluate the performance of different prompt-based methods, including task decomposition, multi-path selection, and reflection, on four benchmarks, demonstrating the potential of LLMs for planning and problem-solving. 
</b>

In [ ]:
check2 = chat_engine.chat("continue the abstract, add one more sentence to the previous two sentence")

In [ ]:
display(Markdown(f"<b>{check2}</b>"))

<b>This paper explores the use of large language models (LLMs) as agents in various environments, including question-answering and interactive programming. The authors evaluate the performance of different prompt-based methods, including task decomposition, multi-path selection, and reflection, on four benchmarks, demonstrating the potential of LLMs for planning and problem-solving.  The paper also discusses the challenges and opportunities of using LLMs as agents, including the need for robust evaluation methods and the development of techniques for improving memory and reasoning capabilities. 
</b>

In [ ]:
check3 = chat_engine.chat("make the above abstract into a poem")

In [ ]:
display(Markdown(f"<b>{check3}</b>"))

<b>In realms of code and language vast,
LLMs as agents, a future cast.
From questions answered, tasks defined,
To worlds of code, their minds entwined.

With prompts they learn, decompose the task,
Through paths they choose, their knowledge amassed.
On benchmarks tested, their prowess shown,
For planning, solving, they've truly grown.

But challenges remain, a path to pave,
To measure their worth, their skills to save.
With memory strong, and reasoning keen,
LLMs as agents, a future unseen. 
</b>